In [ ]:
import gc
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import json
split = "validate"

# Load the JSON data
with open(f'drive/My Drive/PhD/LexDrafter_Paper/dataset/updated_{split}_split_score.json', 'r') as json_file:
    data = json.load(json_file)

In [ ]:
import html
import re
import difflib

threshold = 1.0

# Helper function to clean text
def clean_text(text):
    # Remove HTML tags
    text = html.unescape(text)
    text = re.sub(r'<.*?>', '', text)
    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    # Remove special characters like \u002
    text = re.sub(r'\\u[0-9A-Fa-f]+', '', text)
    return text.strip()

def token_count(text):
  input_tokens = tokenizer(text)["input_ids"]
  input_token_count = len(input_tokens)
  return input_token_count

def are_sentences_similar(sentence1, sentence2):
    sentence1_no_spaces = ''.join(sentence1.split())
    sentence2_no_spaces = ''.join(sentence2.split())
    similarity_ratio = difflib.SequenceMatcher(None, sentence1_no_spaces, sentence2_no_spaces).ratio()
    return similarity_ratio >= threshold

## Definition Generation

In [ ]:
# Common required libraries
!pip install -q einops accelerate langchain bitsandbytes sentencepiece loralib
!pip -q install git+https://github.com/huggingface/transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.3 MB/s eta 0:00:00


### Definition Generation Evaluation

### Prompt Generation

In [ ]:
system_role_prompt = "Act as a Lawyer drafting European Legislative documents to be published on Eur-Lex website."

In [ ]:
system_instruction_prompt = "Define the term: {term}, based on the sentences provided between the triple dashes where different sentences are splitted by new line character \n. ---{sentences}---"

In [ ]:
system_context_prompt = "Provide a clear and concise definition strictly within 25 to 45 words that accurately conveys the meaning within the context of the sentences."

In [ ]:
system_output_prompt = """Give your output in the following JSON format:
{{
  "term": "{term}"
  "definition": "```output text```"
}}

ONLY return the JSON with the keys: [term, definition], do not add ANYTHING, NO INTERPRETATION!"""

In [ ]:
vicuna_template = f"""
{system_role_prompt}\n
{system_instruction_prompt}\n
{system_context_prompt}\n
{system_output_prompt}
"""

### Alpaca Vicuna 7B

#### References:
1. [Alpaca 7B](https://www.youtube.com/watch?v=v6sF8Ed3nTE)

In [ ]:
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain

import torch

Source of latest Vicuna Model:
https://github.com/lm-sys/FastChat/blob/main/docs/vicuna_weights_version.md

In [ ]:
model = "lmsys/vicuna-7b-v1.5"

tokenizer = LlamaTokenizer.from_pretrained(model)

pipeline = pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    do_sample=True,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

Loading the tokenizer from the `special_tokens_map.json` and the `added_tokens.json` will be removed in `transformers 5`,  it is kept for forward compatibility, but it is recommended to update your `tokenizer_config.json` by uploading it again. You will see the new `added_tokens_decoder` attribute that will store the relevant information.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [ ]:
llm = HuggingFacePipeline(pipeline = pipeline,
                          model_kwargs = {'temperature':0.2, 'top_k': 20,'top_p': 0.6, 'repetition_penalty': 1.2, 'max_length': 3000}
                          )
prompt = PromptTemplate(template=vicuna_template, input_variables=["term", "sentences"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
import gc
from tqdm import tqdm

term_list = []

for item in tdqm(data):
  term = item["term"]
  celex_id = item["celex_id"]
  record = f"{term}_{celex_id}"
  if record not in term_list:
    if ((item["existing_record"] == ["NEW TERM"]) and ((item["generated_definition"] == "NO JSON AS AN OUTPUT OBTAINED") or (item["generated_definition"] == ""))):
      scores = item["scores"]
      article_scores = {
        key: value["article_score"]
        for key, value in scores.items()
        # if ("Article" in key and key != "Article 2" and key != "Not Article") or (key == "Article 2" and len(scores.items()) > 1)
        if ("Article" in key and key != "Not Article")
      }

      # If any article exist to provide context then only go ahead otherwise go to next term
      if article_scores:
          max_score = max(article_scores.values())
          selected_articles = [key for key, value in article_scores.items() if value == max_score]

          # Get existing sentences for the selected article
          sentences = item["existing_sentences"].get(selected_articles[0], [])

          # Create a list of tuples where each tuple contains a sentence and its score
          sentence_scores = [(sentence, item["scores"][selected_articles[0]]["statement_scores"]) for sentence in sentences]

          # Sort sentences based on statement_scores in descending order
          sorted_sentences = sorted(
              [(sentence, score) for sentence, score in zip(sentences, item["scores"][selected_articles[0]]["statement_scores"]) if score > 0],
              key=lambda x: x[1], reverse=True
          )
          # Extract cleaned sentences and create a single string with each retrieved sentence on a new line
          cleaned_sentences = [clean_text(sentence) for sentence, _ in sorted_sentences]

          # List to store unique sentences
          unique_sentences = []

          # Iterate through the selected_cleaned_sentences
          for sentence in cleaned_sentences:
            # Flag to indicate if the sentence is a duplicate
            is_duplicate = False

            # Check if the sentence is similar to any existing unique sentence
            for unique_sentence in unique_sentences:
                if are_sentences_similar(sentence, unique_sentence):
                    is_duplicate = True
                    break

            # If not a duplicate, add it to the unique_sentences list
            if not is_duplicate:
                unique_sentences.append(sentence)

          # Since the language model is having the max context length restriction
          total_tokens = 0
          selected_cleaned_sentences = []

          for sentence in unique_sentences:
              sentence_tokens = token_count(sentence)
              if total_tokens + sentence_tokens <= 2500:
                  selected_cleaned_sentences.append(sentence)
                  total_tokens += sentence_tokens
              else:
                  break

          # Remove duplicate sentences if any
          sentences_str = "\n".join(selected_cleaned_sentences)

          vicuna_response = llm_chain.run({"term": term, "sentences": sentences_str})
          print("#########")
          print(f"term: {term}\n")
          print(f"celex_id: {celex_id}\n")
          print(f"response: {vicuna_response}")

          try:
            # Define a regular expression pattern to match the JSON-like structure
            pattern = r'\{[^{}]+\}'
            # Search for the pattern in the text
            match = re.search(pattern, vicuna_response)

            # If a match is found, extract and parse the JSON-like structure
            if match:
              try:
                json_text = match.group()
                json_data = json.loads(json_text)

                # Store the extracted JSON data in a field
                vicuna_response_updated = {
                    "term": json_data.get("term", ""),
                    "definition": json_data.get("definition", "")
                  }
              except json.JSONDecodeError as e:
                # Convert the string value to a JSON object
                json_object = json.loads(vicuna_response)
                # Convert the JSON object to a dictionary
                json_dict = dict(json_object)
                vicuna_response_updated = json_dict

              print(f"updated_response: {vicuna_response_updated}")

              # Concatenate the term and definition
              concatenated_text = f"'{vicuna_response_updated['term']}' means {vicuna_response_updated['definition'].lower()}"
              item["generated_definition"] = concatenated_text.strip()
              output_file_path = f"drive/My Drive/PhD/LexDrafter_Paper/dataset/vicuna/{split}_{term}_{celex_id}.json"
              with open(output_file_path, 'w') as json_file:
                json.dump(item, json_file, indent=2)

              term_list.append(f"{term}_{celex_id}")
              print(f"term_list = {term_list}")
              JSON_file_path = f"drive/My Drive/PhD/LexDrafter_Paper/dataset/vicuna/{split}_term_list_run2.json"
              with open(JSON_file_path, 'w') as json_file:
                json.dump(term_list, json_file, indent=4)

              torch.cuda.empty_cache()
              gc.collect()
            else:
              print("------- Error -------")
              print(f"term: {term}\n")
              print(f"celex_id: {item['celex_id']}\n")
              print(f"response: {vicuna_response}")
              print(f"error: NO JSON AS AN OUTPUT OBTAINED")
              item["generated_definition"] = "NO JSON AS AN OUTPUT OBTAINED"

          except Exception as e:
            print("------- Error -------")
            print(f"term: {term}\n")
            print(f"celex_id: {item['celex_id']}\n")
            print(f"response: {vicuna_response}")
            print(f"error: {e}")
            continue
      else:
        term_list.append(f"{term}_{celex_id}")
        print(f"term_list = {term_list}")
        JSON_file_path = f"drive/My Drive/PhD/LexDrafter_Paper/dataset/vicuna/{split}_term_list_run2.json"
        with open(JSON_file_path, 'w') as json_file:
          json.dump(term_list, json_file, indent=4)
        torch.cuda.empty_cache()
        gc.collect()
    else:
      print(f"NOT A NEW TERM\n")
      term_list.append(f"{term}_{celex_id}")
      print(f"term_list = {term_list}")
      JSON_file_path = f"drive/My Drive/PhD/LexDrafter_Paper/dataset/vicuna/{split}_term_list_run2.json"
      with open(JSON_file_path, 'w') as json_file:
        json.dump(term_list, json_file, indent=4)
      torch.cuda.empty_cache()
      gc.collect()

OutOfMemoryError: ignored

In [ ]:
JSON_file_path = f"drive/My Drive/PhD/LexDrafter_Paper/dataset/vicuna/final_{split}_dataset_run2.json"
with open(JSON_file_path, 'w') as json_file:
  json.dump(data, json_file, indent=4)

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

!nvidia-smi

import torch
# Clear the GPU cache
torch.cuda.empty_cache()
gc.collect()